In [1]:
from dotenv import load_dotenv
import os
import requests 
import pandas as pd

load_dotenv()
api_key = os.environ.get('riot_api_key')

In [ ]:
def get_soloq_leaderboard(top=None):

    root = "https://br1.api.riotgames.com/"

    endpoints = {
        "challenger_endpoint": "lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5",
        "grandmaster_endpoint": "lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5",
        "master_endpoint": "lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
    }

    responses = {}  

    dataframes = {}

    for name, endpoint in endpoints.items():
        response = requests.get(f"{root}{endpoint}?api_key={api_key}")
        responses[f"{name}_response"] = response

    for name, response in responses.items():
        rank_name = name.split('_')[0]
        df = pd.DataFrame(response.json()['entries'])
        dataframes[f'{rank_name}_df'] = df

    leaderboard_df = (
        pd.concat(dataframes.values(), ignore_index=True)
        .drop(columns='rank')  
        .sort_values('leaguePoints', ascending=False)  
        .reset_index(drop=True)  
        .rename_axis('rank').reset_index()  
        .sort_values('rank', ascending=True)  
    )
    
    leaderboard_df['rank'] += 1

    # to-do
    # renomear colunas com underline

    return leaderboard_df

In [44]:
def get_puuid(summoner_id=None):

    root = "https://br1.api.riotgames.com/"
    endpoint = "lol/summoner/v4/summoners/"

    response = requests.get(f'{root}{endpoint}{summoner_id}?api_key={api_key}')

    df_identifiers = pd.DataFrame.from_records([response.json()])[['id', 'accountId', 'puuid', 'summonerLevel']]

    df_identifiers.rename(
        columns={'id':'summonerId'}, 
        inplace=True
    )

    return df_identifiers

In [8]:
result = get_soloq_leaderboard()
result.head()

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,xY7lOHMkLZgA02BYXavriy3ql2vl50uvpF_Af0Fl7jbH,1716,270,216,True,False,False,True
1,2,nttU_Meh80FrSUEwd-ccgC40pDRxob54tqEgSFzy7Ct50u0,1590,208,157,True,False,False,True
2,3,1Dy7Oamd1QXSTUJj5ra6cNezcsqAy_kJlvVh25pJue4DOLU,1331,108,55,False,False,False,False
3,4,1T1VDRlmLEojQhe5LAh6msVNMzRH42sk2_INk-CrpnFBIdU,1313,190,149,True,False,False,True
4,5,krF_JeWAbN5BvYA2VvPLGXe1yHbKPj5bP_QxAmevlHuxu5A,1308,101,52,False,False,False,False


In [45]:
result_puuid = get_puuid(summoner_id='xY7lOHMkLZgA02BYXavriy3ql2vl50uvpF_Af0Fl7jbH')
result_puuid

,summonerId,accountId,puuid,summonerLevel
0,xY7lOHMkLZgA02BYXavriy3ql2vl50uvpF_Af0Fl7jbH,wpu9otPAPa3bsohjBnC1hUaIjIzwj_2k4Umhvcavh0Q,kRacpBZSudVMf0resf3OGlslhRPW86w-X_biJ0pbHsOxP9...,1203
